# Time series figure

This script uses data from cases that have output in chronological order and creates a time series figure

# Import modules

In [ ]:
import sys
#Path to the esm_tools.py script
sys.path.append('/glade/u/home/adamhb/Earth-System-Model-Tools')
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import functools
import cftime
import warnings
from matplotlib.dates import DateFormatter
import math
from datetime import datetime,timedelta
import esm_tools
import esm_viz
import importlib
importlib.reload(esm_tools)
importlib.reload(esm_viz)
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import cftime
import seaborn as sns
pd.set_option('display.max_rows', 500)

In [ ]:
df = pd.read_csv('/glade/work/adamhb/processed_output/supIg105_020224/ts_supIg105_020224.csv',parse_dates=["Date"])
df = df.drop(df.columns[0], axis=1)
vars = list(df.columns[2:])

# vars = ['AGCD',
#  'BA_conifer',
#  'BA_pine',
#  'BA_cedar',
#  'BA_fir',
#  'BA_oak',
#  'TreeStemD',
#  'Pct_shrub_cover_canopy',
#  'Pct_shrub_cover',
#  'Burned_area',
#  'Pct_conifer_cover_canopy',
#  'Pct_oak_cover_canopy',
#  'Combustible_fuel',
#  'Pct_high_severity_1700',
#  'Pct_high_severity_3500']

In [ ]:
df

In [ ]:
df['inst_tag'].unique()

In [ ]:
df.Date

In [ ]:
test_data = df.loc[df['inst_tag'] == 14].dropna()
plt.plot(test_data['Date'],test_data['Pct_high_severity_1700'])

In [ ]:
test_data = df[df['inst_tag'] == 14]
plt.plot(test_data['Date'],test_data['Pct_high_severity_1700'])
#ax.set_xlabel("Year")
plt.show()
print(sum(np.isnan(test_data['Pct_high_severity_1700'].values)))
print(len(test_data['Pct_high_severity_1700'].values))

d = np.array(test_data['Pct_high_severity_1700'].values)

plt.plot(np.array(range(len(d))), d)

In [ ]:
print(len(a.values))

print(sum(np.isnan(a)))

In [ ]:
a = df.groupby("Date")['Pct_high_severity_1700'].mean()
plt.plot(a.index, a.values, color="black", linestyle="--", label="Mean")
plt.show()

In [ ]:
# Set the number of columns for the subplots
num_cols = 2

# Calculate the number of rows needed for subplots
num_rows = (len(vars) + num_cols - 1) // num_cols

# Convert the year 1870 to a datetime object
year_1870 = datetime(1870, 1, 1)
plt.rc('font', size=16)
#default_font_size = 16

# Create a figure with subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(30, 30), dpi=200)
axes = axes.flatten()

df_PHS = df.dropna()

# Loop through each variable and create a subplot
for i, var in enumerate(vars):
    ax = axes[i]

    if (var == "Pct_high_severity_1700") | (var == "Pct_high_severity_3500"):
       sns.lineplot(x="Date", y=var, units="inst_tag", data=df_PHS, ax=ax, color = "grey", alpha = 0.2, estimator=None)
    else:
       sns.lineplot(x="Date", y=var, units="inst_tag", data=df, ax=ax, color = "grey", alpha = 0.2, estimator=None)

    # Set plot title and labels
    if var == "Burned_area":
        ax.set_title("Fractional area burned")
        ax.set_ylabel("Area burned [% simulation area]", fontsize = default_font_size)
        ax.set_ylim(ymin=0)  # Set ymin for the Burned_area panel to be 0
    elif var == "Pct_high_severity_1700":
        ax.set_title("Percent of fires burning \n at high severity")
        ax.set_ylabel("Percent of fires \n high severity", fontsize = default_font_size)
    elif var == "TreeStemD":
        ax.set_title("Tree stem density")
        ax.set_ylabel("Tree stem density [N ha-1]")
    elif var == "Pct_shrub_cover_canopy":
        ax.set_title("Shrub cover")
        ax.set_ylabel("Shrub cover [% ground area]")
    elif var == "BA_conifer":
        ax.set_title("Conifer basal area")
        ax.set_ylabel("Basal area [m2 ha-1]")
    elif var == "BA_oak":
        ax.set_title("Oak basal area")
        ax.set_ylabel("Basal area [m2 ha-1]")
    elif var == "Pct_conifer_cover_canopy":
        ax.set_title("Conifer cover")
        ax.set_ylabel("Conifer cover [% ground area]")
    elif var == "Pct_oak_cover_canopy":
        ax.set_title("Oak cover")
        ax.set_ylabel("Oak cover [% ground area]")

    ax.set_xlabel("Year")  # Change the x-axis label to "Year"
     # Add a vertical line at 1870 to all panels
    ax.axvline(x=year_1870, color="red", linestyle="--")

    # Add a black trend line representing the mean of the groups


    if (var == "Pct_high_severity_1700") | (var == "Pct_high_severity_3500"):
       mean_values = df_PHS.groupby("Date")[var].mean()
    else:
       mean_values = df.groupby("Date")[var].mean()

    ax.plot(mean_values.index, mean_values.values, color="black", linestyle="--", label="Mean")

    # Remove the legend
    #ax.get_legend().remove()

# Remove any empty subplots
for i in range(len(vars), num_rows * num_cols):
    fig.delaxes(axes[i])

# Adjust layout
plt.tight_layout()

#plt.savefig(figName, format="pdf", bbox_inches="tight")

# Show the plot
plt.show()

# Functions

In [ ]:
def running_mean(arr, window_size):
    """
    Calculate the running mean of a NumPy array.

    Parameters:
    - arr: NumPy array
        The input array for which you want to calculate the running mean.
    - window_size: int
        The size of the window used to compute the mean.

    Returns:
    - mean_arr: NumPy array
        An array containing the running mean values.
    """
    if window_size <= 0:
        raise ValueError("Window size must be a positive integer.")

    mean_arr = np.zeros_like(arr, dtype=float)  # Initialize the result array with zeros
    cumsum = np.cumsum(arr)  # Calculate the cumulative sum of the input array

    for i in range(len(arr)):
        if i < window_size - 1:
            # For the first few elements, use a smaller window if not enough data is available
            mean_arr[i] = cumsum[i] / (i + 1)
        else:
            # For the rest of the elements, use a window of size 'window_size'
            mean_arr[i] = (cumsum[i] - cumsum[i - window_size]) / window_size

    return mean_arr

def convert_cftime_to_datetime(cftime_obj):

    # Convert it to a datetime object
    datetime_obj = datetime(cftime_obj.year, cftime_obj.month, cftime_obj.day,
                            cftime_obj.hour, cftime_obj.minute, cftime_obj.second)

    return datetime_obj

# Setup

In [ ]:
# Root path where case output lives
model_output_root = '/glade/derecho/scratch/adamhb'

pft_names = ["pine","cedar","fir","shrub","oak"]
pft_colors = ['gold','darkorange','darkolivegreen','brown','springgreen']
n_pfts = len(pft_names)



# nignition = 0.055


#cases = ['CZ2_2015_2099_ssp370_012624_-17e2acb6a_FATES-5b076b69']

#figName = "/glade/u/home/adamhb/california-fates/figures/response_to_fire_012924.pdf"

# cases = ['CZ2_equilibrium_011824_-17e2acb6a_FATES-5b076b69',
#          'CZ2_trans_1870_1951_011924_-17e2acb6a_FATES-5b076b69',
#          'CZ2_trans_1951_2020_012124_-17e2acb6a_FATES-5b076b69']

#tags = ['0009','0005','0047','0002','0035','0050','0010']


# tags = ['0002',
#  '0004',
#  '0008',
#  #'0013',
#  '0014',
#  '0017',
#  #'0021',
#  '0024',
#  '0025',
#  '0026',
#  '0030',
#  #'0046',
#  '0052']#,
#  #'0001','0003','0050'] # These need to be added in because they have oaks doing well and we need more oak-dominant ensemble members



# nignition = 0.0825


figName = "/glade/u/home/adamhb/california-fates/figures/response_to_fire_012924_nignition_50pctup.pdf"

cases = ['CZ2_equilibrium_011824_-17e2acb6a_FATES-5b076b69',
         'CZ2_trans_1870_1951_012624_-17e2acb6a_FATES-5b076b69',
         'CZ2_trans_1951_2020_012824_-17e2acb6a_FATES-5b076b69']

tags = ['0002',
 '0008',
 '0013',
 '0014',
 '0021',
 '0024',
 '0025',
 '0026',
 '0030',
 '0037',
 '0041',
 '0046',
 '0052',
 '0053']


tags_int = [int(t) for t in tags]

timeseries_path = '/glade/work/adamhb/data_for_figures/CZ2_timeseries_012924_nignition_50pctup.csv'
timeseries_path_PHS = '/glade/work/adamhb/data_for_figures/CZ2_timeseries_PHS_012924_nignition_50pctup.csv'

# Define the list of variables to plot in the desired order
variables = ["Burned_area",
             "Pct_high_severity_3500","Pct_high_severity_1700",
             "TreeStemD", "Pct_shrub_cover_canopy", "BA_conifer", "BA_oak",
             "Pct_conifer_cover_canopy","Pct_oak_cover_canopy"]

# Load data

In [ ]:
full_time_series_fields = [
          #have on to import these dimensions
          'FATES_SEED_PROD_USTORY_SZ',
          'FATES_VEGC_AP',
          #patches and cohorts
          'FATES_NPATCHES',
          'FATES_PATCHAREA_AP',
          'FATES_NPATCH_AP',
          #structure
          #'FATES_LAI_AP',
          #density
          'FATES_NPLANT_PF',
          'FATES_NPLANT_SZAPPF',
          'FATES_NPLANT_SZPF',
          #basal area
          'FATES_BASALAREA_SZPF',
          #crown_area
          'FATES_CANOPYCROWNAREA_PF',
          'FATES_CANOPYCROWNAREA_APPF',
          'FATES_CROWNAREA_APPF',
          'FATES_CROWNAREA_PF',
          #biomass
          #'FATES_VEGC_PF','FATES_VEGC_AP','FATES_VEGC_ABOVEGROUND','FATES_VEGC_ABOVEGROUND_SZPF',
          #growth
          #'FATES_DDBH_SZPF',
          #'FATES_DDBH_CANOPY_SZAP','FATES_DDBH_USTORY_SZAP',
          #mortality
          'FATES_MORTALITY_PF',
          #'FATES_MORTALITY_CANOPY_SZAP','FATES_MORTALITY_USTORY_SZAP',
          'FATES_MORTALITY_BACKGROUND_SZPF','FATES_MORTALITY_HYDRAULIC_SZPF','FATES_MORTALITY_CSTARV_SZPF',
          'FATES_MORTALITY_FIRE_SZPF','FATES_MORTALITY_CROWNSCORCH_SZPF',
          'FATES_MORTALITY_SENESCENCE_SZPF','FATES_MORTALITY_TERMINATION_SZPF','FATES_MORTALITY_LOGGING_SZPF',
          'FATES_MORTALITY_FREEZING_SZPF','FATES_MORTALITY_AGESCEN_SZPF','FATES_MORTALITY_IMPACT_SZPF',
          #seed production and recruitment
          #GPP and NPP
          'FATES_NPP_PF','FATES_NPP_SZPF',
          'FATES_RECRUITMENT_PF',
          #'FATES_AUTORESP_SZPF','FATES_MAINTAR_SZPF',
          #physical environment
          'SMP',
          #allocation
          #'FATES_STOREC_CANOPY_SZPF','FATES_STOREC_USTORY_SZPF',
          #fire
          'FATES_FUEL_AMOUNT_APFC',
          'FATES_BURNFRAC','FATES_IGNITIONS','FATES_FIRE_INTENSITY_BURNFRAC',
          'FATES_FUEL_BULKD','FATES_FUEL_SAV',
          'FATES_FUEL_AMOUNT_AP',
          'FATES_FIRE_INTENSITY_BURNFRAC_AP',
          'FATES_BURNFRAC_AP',
          'FATES_NESTEROV_INDEX',
          'FATES_VEGC_ABOVEGROUND'
          ]

# Process data

In [ ]:
def get_df_of_data(case,tag):
    
    print("Working on",case,"-",tag)
    
    # equilibrium_to_TRANS_df = pd.DataFrame({'eq_tag':['0049','0054','0062','0023','0011','0012','0033'],
    #                              'trans_tag':['0009','0005','0047','0002','0035','0050','0010']})
    
    
    if "equilibrium" in case:
        # days_offset = -430 * 365.25
        # print(case)
        # print("offset",days_offset)
        
        years = list(range(1850, 1870))
        
        #trans_tag = list(equilibrium_to_TRANS_df.loc[equilibrium_to_TRANS_df['eq_tag'] == tag]['trans_tag'])[0]
        
    elif "trans" in case:
        # days_offset = 0
        # trans_tag = tag
        years = list(range(1870,1951))
        
        if '1951_2020' in case:
            years = list(range(1951,2020))

    elif "ssp" in case:
        years = list(range(2015,2098))
    
    #offset = timedelta(days=abs(days_offset))
    
    
    ds = esm_tools.load_fates_output_data(model_output_root=model_output_root,
                            case_name = case,
                            years = years,
                            fields = full_time_series_fields,
                            inst_tag = tag,
                            manual_path = None)
    
    dates = [convert_cftime_to_datetime(ds.time.values[i]) for i in range(len(ds.time.values))]
    
    # if days_offset < 0:
    #     dates = [date - offset for date in dates]
    # else:
    #     dates = [date + offset for date in dates]

    my_variables = ["Harmonized_tag","Date","BA_conifer","BA_oak","TreeStemD","Pct_shrub_cover_canopy",
                    "Pct_shrub_cover","Burned_area","Pct_conifer_cover_canopy","Pct_oak_cover_canopy"]
    my_dict = {}
    for i in my_variables:
        my_dict[i] = None

    my_dict['Harmonized_tag'] = [tag] * len(dates)               
    my_dict["Date"] = dates
    my_dict['BA_conifer'] = esm_tools.get_conifer_basal_area_over_time(ds,dbh_min=10)
    my_dict['BA_oak'] = esm_tools.get_oak_basal_area_over_time(ds,dbh_min = 10)
    my_dict['TreeStemD'] = esm_tools.get_total_stem_den(ds,trees_only=True,dbh_min=10,over_time=True)
    my_dict['Pct_shrub_cover'] = esm_tools.get_pft_level_crown_area(ds,pft_index = 3,canopy_area_only = False,over_time=True)
    my_dict['Pct_shrub_cover_canopy'] = esm_tools.get_pft_level_crown_area(ds,pft_index = 3,canopy_area_only = True,over_time=True)
    my_dict['Pct_oak_cover_canopy'] = esm_tools.get_pft_level_crown_area(ds,pft_index = 4,canopy_area_only = True,over_time=True)
    my_dict['Pct_conifer_cover_canopy'] = esm_tools.get_conifer_crown_area(ds,canopy_area_only = True, over_time = True)
    burn_frac = esm_tools.get_mean_annual_burn_frac(ds,over_time=True)
    my_dict['Burned_area'] = running_mean(burn_frac, 36)

    # Get the running mean of PHS
    window = 36
    iterations = len(ds.time) // window
    PHS_dates = []
    PHS_3500 = []
    PHS_1700 = []
    for i in range(iterations):
        start_time_index = i * window
        end_time_index = min(start_time_index + 36, len(ds.time))
        mid_time_index = start_time_index + (window // 2)
        #print(start_time_index,"-",end_time_index)
        mid_date = convert_cftime_to_datetime(ds.time.values[mid_time_index])
        
        # if days_offset < 0:
        #     mid_date = mid_date - offset
        # else:
        #     mid_date = mid_date + offset
        
        PHS_dates.append(mid_date)
        PHS_3500.append(esm_tools.get_PHS_FLI_thresh_isel(ds,start_time_index,end_time_index,3500))
        PHS_1700.append(esm_tools.get_PHS_FLI_thresh_isel(ds,start_time_index,end_time_index,1700))

    PHS_dict = {}
    PHS_dict['Date']= PHS_dates
    PHS_dict['Harmonized_tag']= [tag] * len(PHS_dates)  
    PHS_dict['Pct_high_severity_1700'] = PHS_1700
    PHS_dict['Pct_high_severity_3500'] = PHS_3500
    
    #print(my_dict)    
    df = pd.DataFrame(my_dict)
    df_PHS = pd.DataFrame(PHS_dict)
    
    return df, df_PHS

In [ ]:
ds = esm_tools.load_fates_output_data(model_output_root=model_output_root,
                            case_name = 'CZ2_trans_1951_2020_012824_-17e2acb6a_FATES-5b076b69',
                            years = list(range(1960,1966)),
                            fields = full_time_series_fields,
                            inst_tag = "0001",
                            manual_path = None)

In [ ]:
df = esm_tools.get_ts('CZ2_trans_1951_2020_012824_-17e2acb6a_FATES-5b076b69',
                              list(range(1960,1970)),"0008")

In [ ]:
df_all = pd.merge(df,df_PHS,on="Date",how = "left")

In [ ]:
def get_df_of_data(case,years,tag):
    
    print("Working on",case,"-",tag)
    
    ds = esm_tools.load_fates_output_data(model_output_root=model_output_root,
                            case_name = case,
                            years = years,
                            fields = full_time_series_fields,
                            inst_tag = tag,
                            manual_path = None)
    
    dates = [convert_cftime_to_datetime(ds.time.values[i]) for i in range(len(ds.time.values))]
    
    # if days_offset < 0:
    #     dates = [date - offset for date in dates]
    # else:
    #     dates = [date + offset for date in dates]

    my_variables = ["Harmonized_tag","Date","BA_conifer","BA_oak","TreeStemD","Pct_shrub_cover_canopy",
                    "Pct_shrub_cover","Burned_area","Pct_conifer_cover_canopy","Pct_oak_cover_canopy"]
    my_dict = {}
    for i in my_variables:
        my_dict[i] = None

    my_dict['Harmonized_tag'] = [tag] * len(dates)               
    my_dict["Date"] = dates
    my_dict['BA_conifer'] = esm_tools.get_conifer_basal_area_over_time(ds,dbh_min=10)
    my_dict['BA_oak'] = esm_tools.get_oak_basal_area_over_time(ds,dbh_min = 10)
    my_dict['TreeStemD'] = esm_tools.get_total_stem_den(ds,trees_only=True,dbh_min=10,over_time=True)
    my_dict['Pct_shrub_cover'] = esm_tools.get_pft_level_crown_area(ds,pft_index = 3,canopy_area_only = False,over_time=True)
    my_dict['Pct_shrub_cover_canopy'] = esm_tools.get_pft_level_crown_area(ds,pft_index = 3,canopy_area_only = True,over_time=True)
    my_dict['Pct_oak_cover_canopy'] = esm_tools.get_pft_level_crown_area(ds,pft_index = 4,canopy_area_only = True,over_time=True)
    my_dict['Pct_conifer_cover_canopy'] = esm_tools.get_conifer_crown_area(ds,canopy_area_only = True, over_time = True)
    burn_frac = esm_tools.get_mean_annual_burn_frac(ds,over_time=True)
    my_dict['Burned_area'] = running_mean(burn_frac, 36)

    # Get the running mean of PHS
    window = 36
    iterations = len(ds.time) // window
    PHS_dates = []
    PHS_3500 = []
    PHS_1700 = []
    for i in range(iterations):
        start_time_index = i * window
        end_time_index = min(start_time_index + 36, len(ds.time))
        mid_time_index = start_time_index + (window // 2)
        #print(start_time_index,"-",end_time_index)
        mid_date = convert_cftime_to_datetime(ds.time.values[mid_time_index])
        
        # if days_offset < 0:
        #     mid_date = mid_date - offset
        # else:
        #     mid_date = mid_date + offset
        
        PHS_dates.append(mid_date)
        PHS_3500.append(esm_tools.get_PHS_FLI_thresh_isel(ds,start_time_index,end_time_index,3500))
        PHS_1700.append(esm_tools.get_PHS_FLI_thresh_isel(ds,start_time_index,end_time_index,1700))

    PHS_dict = {}
    PHS_dict['Date']= PHS_dates
    PHS_dict['Harmonized_tag']= [tag] * len(PHS_dates)  
    PHS_dict['Pct_high_severity_1700'] = PHS_1700
    PHS_dict['Pct_high_severity_3500'] = PHS_3500
    
    #print(my_dict)    
    df = pd.DataFrame(my_dict)
    df_PHS = pd.DataFrame(PHS_dict)
    
    return df, df_PHS

# Apply

In [ ]:
output_df = pd.DataFrame()
output_df_PHS = pd.DataFrame()
for c in cases:
    for t in tags:
        tmp_df, tmp_df_PHS = get_df_of_data(c,t)
        output_df = pd.concat([output_df,tmp_df],axis = 0)
        output_df_PHS = pd.concat([output_df_PHS,tmp_df_PHS],axis = 0)

#output_df
output_df_PHS.to_csv(timeseries_path_PHS)
output_df.to_csv(timeseries_path)

# Make figures

In [ ]:
# Replace 'your_file.csv' with the actual CSV file path

## Load future data
# output_df = pd.read_csv(timeseries_path, parse_dates=['Date'], date_parser=pd.to_datetime)
# output_df = output_df.loc[output_df['Harmonized_tag'].isin(tags_int)]

# output_df_PHS = pd.read_csv(timeseries_path_PHS, parse_dates=['Date'], date_parser=pd.to_datetime)
# output_df_PHS = output_df_PHS[output_df_PHS['Harmonized_tag'].isin(tags_int)]


# ## Load historical data
# timeseries_path_PHS_hist = '/glade/work/adamhb/data_for_figures/CZ2_timeseries_PHS_012424.csv'
# timeseries_path_hist = '/glade/work/adamhb/data_for_figures/CZ2_timeseries_012424.csv'

# output_df_PHS_hist = pd.read_csv(timeseries_path_PHS_hist, parse_dates=['Date'], date_parser=pd.to_datetime)
# output_df_PHS_hist = output_df_PHS_hist.loc[output_df_PHS_hist['Harmonized_tag'].isin(tags_int)]
# # Filter to before 2015 (when projections start)
# output_df_PHS_hist = output_df_PHS_hist.loc[output_df_PHS_hist['Date'] < '2015-01-01']

# output_df_hist = pd.read_csv(timeseries_path_hist, parse_dates=['Date'], date_parser = pd.to_datetime)
# output_df_hist = output_df_hist.loc[output_df_hist['Harmonized_tag'].isin(tags_int)]
# output_df_hist = output_df_hist.loc[output_df_hist['Date'] < '2015-01-01']

# # Join
# output_df = pd.concat([output_df_hist,output_df],axis = 0)
# output_df_PHS = pd.concat([output_df_PHS_hist,output_df_PHS],axis = 0)

In [ ]:
# output_df_PHS

# np.isnan(output_df_PHS.groupby("Date")['Pct_high_severity_3500'])

## Histogram of 1984-2009 burned area

In [ ]:
# output_df.loc[output_df['Date'] >= '2010-01-01'].groupby('Harmonized_tag')['Burned_area'].mean().hist()
# plt.xlabel("Burned area")
# plt.ylabel("number of ensemble members")
# plt.show()

data = output_df.loc[(output_df['Date'] >= '1984-01-01') & (output_df['Date'] <= '2009-01-01')].groupby('Harmonized_tag')['Burned_area'].mean()
plt.hist(data, bins=10, alpha=0.5, color='blue', edgecolor='black')

plt.axvline(x=0.0053487, color='black', linestyle='--', linewidth=2)
plt.xlabel("Burned area")
plt.ylabel("number of ensemble members")
plt.show()

## Histogram of 2010-2020 burned area

In [ ]:
# output_df.loc[output_df['Date'] >= '2010-01-01'].groupby('Harmonized_tag')['Burned_area'].mean().hist()
# plt.xlabel("Burned area")
# plt.ylabel("number of ensemble members")
# plt.show()

data = output_df.loc[output_df['Date'] >= '2010-01-01'].groupby('Harmonized_tag')['Burned_area'].mean()
plt.hist(data, bins=10, alpha=0.5, color='blue', edgecolor='black')

plt.axvline(x=0.02, color='black', linestyle='--', linewidth=2)
plt.xlabel("Burned area")
plt.ylabel("number of ensemble members")
plt.show()

## Histogram of 2010-2020 PHS-3500

In [ ]:
# output_df_PHS.loc[output_df_PHS['Date'] >= '2010-01-01'].groupby('Harmonized_tag')['Pct_high_severity_3500'].mean().hist()
# plt.xlabel("PHS-3500")
# plt.ylabel("number of ensemble members")
# plt.show()

data = output_df_PHS.loc[output_df_PHS['Date'] >= '2010-01-01'].groupby('Harmonized_tag')['Pct_high_severity_3500'].mean()
plt.hist(data, bins=10, alpha=0.5, color='blue', edgecolor='black')
plt.axvline(x=43, color='black', linestyle='--', linewidth=2)
plt.xlabel("PHS-3500")
plt.ylabel("number of ensemble members")
plt.show()

## Histogram of 1985 PHS-3500

In [ ]:
# output_df_PHS.loc[output_df_PHS['Date'] >= '2010-01-01'].groupby('Harmonized_tag')['Pct_high_severity_3500'].mean().hist()
# plt.xlabel("PHS-3500")
# plt.ylabel("number of ensemble members")
# plt.show()

data = output_df_PHS.loc[(output_df_PHS['Date'] <= '2010-01-01') & (output_df_PHS['Date'] >= '1950-01-01')].groupby('Harmonized_tag')['Pct_high_severity_3500'].mean()
plt.hist(data, bins=10, alpha=0.5, color='blue', edgecolor='black')
plt.axvline(x=25, color='black', linestyle='--', linewidth=2)
plt.xlabel("PHS-3500")
plt.ylabel("number of ensemble members")
plt.show()

## Histogram of 2010 conifer basal area

In [ ]:
data = output_df.loc[(output_df['Date'] >= '1995-01-01') & (output_df['Date'] <= '2015-01-01')].groupby('Harmonized_tag')['BA_conifer'].mean()
plt.hist(data, bins=10, alpha=0.5, color='blue', edgecolor='black')

plt.axvline(x=49, color='black', linestyle='--', linewidth=2)
plt.xlabel("BA conifer")
plt.ylabel("number of ensemble members")
plt.show()

In [ ]:
output_df[['Date','BA_conifer']]

In [ ]:
data = output_df.loc[(output_df['Date'] >= '2015-01-01') & (output_df['Date'] <= '2050-01-01')].groupby('Harmonized_tag')['BA_conifer'].mean()
plt.hist(data, bins=10, alpha=0.5, color='blue', edgecolor='black')

plt.axvline(x=49, color='black', linestyle='--', linewidth=2)
plt.xlabel("BA conifer")
plt.ylabel("number of ensemble members")
plt.show()

## Histogram of 1985 shrub cover

In [ ]:
data = output_df.loc[(output_df['Date'] >= '1965-01-01') & (output_df['Date'] <= '2000-01-01')].groupby('Harmonized_tag')['Pct_shrub_cover_canopy'].mean()
plt.hist(data, bins=10, alpha=0.5, color='blue', edgecolor='black')

plt.axvline(x=0.22, color='black', linestyle='--', linewidth=2)
plt.xlabel("Burned area")
plt.ylabel("number of ensemble members")
plt.show()

### Forest metrics

In [ ]:
# Set the number of columns for the subplots
num_cols = 2

# Calculate the number of rows needed for subplots
num_rows = (len(variables) + num_cols - 1) // num_cols

# Create a figure with subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(16.8, 16.8), dpi=400)
axes = axes.flatten()

# Loop through each variable and create a subplot

# Convert the year 1870 to a datetime object
year_1870 = datetime(1870, 1, 1)
plt.rc('font', size=16)

default_font_size = 16
for i, var in enumerate(variables):
    ax = axes[i]

    if (var == "Pct_high_severity_1700") | (var == "Pct_high_severity_3500"):
        print(output_df_PHS[output_df_PHS[['Date','Harmonized_tag']].duplicated()])
        sns.lineplot(x="Date", y=var, units="Harmonized_tag", data=output_df_PHS.reset_index(), ax=ax, color = "grey", alpha = 0.2, estimator=None)
    else:
    # Use seaborn to create a line plot for each unique group with grey lines
        sns.lineplot(x="Date", y=var, units="Harmonized_tag", data=output_df, ax=ax, color = "grey", alpha = 0.2, estimator=None)

    # Set plot title and labels
    if var == "Burned_area":
        ax.set_title("Fractional area burned")
        ax.set_ylabel("Area burned [% simulation area]", fontsize = default_font_size)
        ax.set_ylim(ymin=0)  # Set ymin for the Burned_area panel to be 0
    elif var == "Pct_high_severity_1700":
        ax.set_title("Percent of fires burning \n at high severity")
        ax.set_ylabel("Percent of fires \n high severity", fontsize = default_font_size)
    elif var == "TreeStemD":
        ax.set_title("Tree stem density")
        ax.set_ylabel("Tree stem density [N ha-1]")
    elif var == "Pct_shrub_cover_canopy":
        ax.set_title("Shrub cover")
        ax.set_ylabel("Shrub cover [% ground area]")
    elif var == "BA_conifer":
        ax.set_title("Conifer basal area")
        ax.set_ylabel("Basal area [m2 ha-1]")
    elif var == "BA_oak":
        ax.set_title("Oak basal area")
        ax.set_ylabel("Basal area [m2 ha-1]")
    elif var == "Pct_conifer_cover_canopy":
        ax.set_title("Conifer cover")
        ax.set_ylabel("Conifer cover [% ground area]")
    elif var == "Pct_oak_cover_canopy":
        ax.set_title("Oak cover")
        ax.set_ylabel("Oak cover [% ground area]")

    ax.set_xlabel("Year")  # Change the x-axis label to "Year"
     # Add a vertical line at 1870 to all panels
    ax.axvline(x=year_1870, color="red", linestyle="--")

    # Add a black trend line representing the mean of the groups
    
    
    if (var == "Pct_high_severity_1700") | (var == "Pct_high_severity_3500"):
        mean_values = output_df_PHS.groupby("Date")[var].mean()
    else:
        mean_values = output_df.groupby("Date")[var].mean()
    
    ax.plot(mean_values.index, mean_values.values, color="black", linestyle="--", label="Mean")

    # Remove the legend
    #ax.get_legend().remove()

# Remove any empty subplots
for i in range(len(variables), num_rows * num_cols):
    fig.delaxes(axes[i])

# Adjust layout
plt.tight_layout()

#plt.savefig(figName, format="pdf", bbox_inches="tight")

# Show the plot
plt.show()

### PHS

In [ ]:
# sns.lineplot(x="Date", y="Pct_high_severity_1700", data=output_df_PHS, color = "black",linestyle = "--")
# plt.ylabel("Percent of fires burning \n at high severity")
# plt.show()